## 분장 분리(마침표를 기준으로 문장 분리) 및 전처리(정규표현식으로 진행)

In [21]:
import re
import json
from importlib.resources import contents

import pymysql
import re
from ekonlpy.tag import Mecab

#문장분리기
def sen_split(text):
    str_text=str(text)
    # type(str_text)
    #print(str_text)
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', str_text)
    return sentences

#정규 표현식으로 필요없는 부분 제거
def tokenize_korean_text(text):
    # #영어 제거
    # pattern = '[a-zA-Z]'
    # text = re.sub(pattern, '', text)
    
    # E-mail 제거
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
    text = re.sub(pattern, '', text)
    
    # URL 제거
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text = re.sub(pattern, '', text)
    
    # HTML 태그 제거
    pattern = '<[^>]*>'
    text = re.sub(pattern, '', text)
    
    # 한글 자음, 모음 제거
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'
    text = re.sub(pattern, '', text)
    
    # 주소 관련 패턴 제거
    pattern = '([가-힣A-Za-z·\d~\-\.]{2,}(로|길)+)'
    text = re.sub(pattern, '', text)
    
    pattern = '([가-힣A-Za-z·\d~\-\.]+(읍|동|번지|시|구)+)'
    text = re.sub(pattern, '', text)
    
    pattern = '([가-힣A-Za-z]+(구)\s*[가-힣A-Za-z]+(동))'
    text = re.sub(pattern, '', text)
    
    pattern = '([가-힣a-zA-Z\d]+(아파트|빌라|빌딩|마을))'
    text = re.sub(pattern, '', text)
    
    #특수문자 제거
    pattern= '[`~!@#$%^&*()_|+\-=?;:,.<>\{\}\[\]\\\/]'
    text = re.sub(pattern, '', text)
    
    # 숫자 제거
    pattern = '[0-9]'
    text = re.sub(pattern, '', text)
    return text

#각 문서에 대해 문장 분리 및 정규표현식 진행
def start_split_sentences(text):
    sentences = sen_split(text)
    result=[tokenize_korean_text(sentence) for sentence in sentences]
    return result

#데이터 베이스 접근 및 전처리 진행
class DatabaseManager:
    def __init__(self, host, port, user, password, db_name, charset="utf8"):
        # 데이터베이스 연결 설정
        self.connection = pymysql.connect(
            host=host,
            port=port,
            user=user,
            passwd=password,
            db=db_name,
            charset=charset
        )

    def fetch_sentences(self, limit=10):
        """주어진 쿼리로 데이터를 가져오는 함수"""
        cursor = self.connection.cursor()
        sql = f"SELECT sentences FROM text_analysis WHERE document_type = 'analyst_report' LIMIT {limit};"
        cursor.execute(sql)
        results = cursor.fetchall()
        cursor.close()
        return results

    def close(self):
        """데이터베이스 연결 닫기"""
        self.connection.close()

def main():
    # 데이터베이스 연결 정보
    db_manager = DatabaseManager(
        host='paper-project-1.cne82okq8qfy.us-east-2.rds.amazonaws.com',
        port=3306,
        user="choi",
        password="UbcFp1C!7^shuxKYv4Q$",
        db_name="PAPER_PROJECT"
    )

    try:
        #SQL 쿼리 실행 및 데이터 가져오기
        read = db_manager.fetch_sentences(limit=10)

        # 10개의 데이터를 처리
        for i in range(len(read)):#가져온 데이터 수만큼 반복
            try:
                d = json.loads(read[i][0])
                text = d.get('keyword', '')  # 'keyword' 키가 없으면 빈 문자열을 반환
                sentences=start_split_sentences(text)
                # JSON 변환
                contents={'keyword':sentences}
                return contents
            except (IndexError, json.JSONDecodeError) as e:
                print(f"Error processing document {i+1}: {e}")
    finally:
        db_manager.close()

# main 함수 실행
if __name__ == "__main__":
    contents=main()
    print(contents)


{'keyword': ["'  n¿ù°£Ã¤±ÇÅõÀÚnÃ¤±Ç½ÃÀåÀü¸Án±ÝÀ¶½ÃÀåÂ÷Æ®ºÏnÓßëë°úùÚÍ£' '월간채권투자  월 n n 對應과 限界 n미국에서는 여전히 연준은 물론 정부와 의회  신용경색 해소를 위해 다방면의 노력을 하고 있다", 'n하지만 그것은 미 스템의 붕괴는 모면케 해 줄 수 있을지언정 순환적 경기침체까지 피해가게 만들 수는 없을 것 같다', '장의 성이 보강되는 가운데에서도 실물경기는  가는 상황이라면 이는 국내 금리에도 하락 여건일 수 있다고 본다', ' n이미 정책이 진행중인 미국과 달리 국내에서는 향후 정책의 방향성을 놓고 당국간 입장차가 확연히 n드러나고 있는 상황이다', '월 금통위에서도 그다지 금리 우호적 발언이 나오지 않을 리스크를 경계하고 있는데 하지만 시장 충격이 그다지 크지는 않을  보고 있다', ' 관건은 객관적 환경이 에게 유리하게 전개되느냐에 있을 것이기 때문이다', '물론 우리는 여전히 시간은 점차 한은 편이 아닐 가능성이 우세하다고 보고 있다', 'n           n이머징 신용위험 약간 둔화되는 가운데 한국 외화 조달 비용 다소 낮아져 n   n자료 Bloomberg n  n서철수   n윤여삼   n윤일광   n nContent n對應과 限界  n', '동향 n', '채권투자환경 ', "요약과 전망 장 차트북 n국내금리 동향 국내금리 주요 스프레드 채권수급 n– 장 n채권수급 – 장 벌 금리 동향 상품 지표  환율 주식 시장  산 시장 리스크  인플레 지표 자금 시장 동향 통화량 지표  기술적 지표 이슈 nn       nnnCRS년좌nEMBI 스프레드축반전우bp ' '월간채권투자  월 n n n ", '동향 n강세 흐름 자체는 유지되었으나 환율 및 정책 리스크 부각되며 금리 성 심화 n월은 전반적인 금리 레벨이 높아진 가운데 다양한  인해 금리 성이 재차 확대된 한달이었다', '월 단기급락 이후 월에는 성 축소 양상을 보이던 장은 월에 인플레  환율  장  외국인  기획재정부 vs 한

## 품사태깅 
- 이전에 ekonlpy를 설치해야 함.
- git clone https://github.com/entelecheia/eKoNLPy.git
- cd eKoNLPy
- pip install .
반드시 konlpy와 Mecab이 설치되어 있는지 ekonlpy 설치 전 확인 해야 함.

In [30]:
#문장에서 명사만 추출
def extract_nouns_from_sentences(contents):
    mecab = Mecab()
    res = []
    stopwords = ['NNG', 'NNP']  #일반 명사(NNG), 고유 명사(NNP)만 추출

    for text in contents:  #contents는 문장의 리스트 ([[문장1], [문장2], ...])
        tokens = mecab.pos(text)  #형태소 분석
        tokens = mecab.replace_synonyms(tokens)  #동의어 치환
        tokens = mecab.lemmatize(tokens)  #원형 복원
        
        # 명사만 추출
        nouns = [t[0] for t in tokens if t[1] in stopwords]
        res.append(nouns)  # 문장별로 명사만 추출하여 결과 리스트에 추가

    return res

# 불러오기
result = extract_nouns_from_sentences(contents['keyword'])
print(result[-1])  # 마지막 문장의 명사 리스트 출력


['조사', '자료', '지적재산권', '당사', '당사', '허락', '무단', '복제', '배포']


In [33]:
#불용어 생성
stop_words = ['층','n','마산','가','를','을','점','관','월','일','층','텔','남','좌','리','중','정','미','종']

#불용어 처리
def remove_stopwords(result, stop_words):
    filtered_words = []
    for word in result:
        if len(word)>1 and word not in stop_words:
            filtered_words.append(word)
    return filtered_words

#불러오기
all_tokens=[]
for i in range(len(result)):
    all_tokens.append(remove_stopwords(result[i], stop_words))
all_tokens

[['월간',
  '채권투자',
  '對應',
  '限界',
  '미국',
  'fed',
  '정부',
  '의회',
  '신용경색',
  '해소',
  '다방면',
  '노력'],
 ['스템', '붕괴', '모면', '경기', '침체'],
 ['장의', '보강', '가운데', '실물', '경기', '상황', '라면', '국내', '금리', '하락', '여건'],
 ['정책', '진행', '미국', '국내', '정책', '방향', '당국', '입장차', '상황'],
 ['금통위', '금리', '발언', '위험', '경계', '시장', '충격'],
 ['관건', '객관', '환경', '유리', '전개'],
 ['시간', '한은', '가능성', '우세'],
 ['신흥국',
  '신용위험',
  '둔화',
  '가운데',
  '한국',
  '외화',
  '조달',
  '비용',
  '자료',
  '서철수',
  '윤여',
  '윤일광',
  '對應',
  '限界'],
 ['동향'],
 ['채권투자', '환경'],
 ['요약',
  '전망',
  '차트',
  '국내',
  '금리',
  '동향',
  '국내',
  '금리',
  '주요',
  '스프레드',
  '채권',
  '수급',
  '채권',
  '수급장',
  '금리',
  '동향',
  '상품',
  '지표',
  '환율',
  '주식시장',
  '시장리스크',
  '인플레이션',
  '지표',
  '자금시장',
  '동향',
  '통화',
  '지표',
  '지표',
  '이슈',
  '스프레드',
  '전우',
  '월간',
  '채권투자'],
 ['동향',
  '강세',
  '흐름',
  '자체',
  '유지',
  '환율',
  '정책',
  '위험',
  '부각',
  '금리',
  '심화',
  '전반',
  '금리',
  '레벨',
  '가운데',
  '다양',
  '인해',
  '금리',
  '확대'],
 ['단기',
  '급락',
  '이후',
  '축소',
  '양상',
  '인플레이션

In [34]:
from gensim.models.ldamodel import LdaModel
from gensim import corpora

#사전 구축
dictionary = corpora.Dictionary(all_tokens) # 토큰 단어와 gensim 내부 아이디 매칭
dictionary.filter_extremes(no_below=2, no_above=0.5) # 빈도 2이상 포함, 전체 50% 이상 단어 제거
corpus = [dictionary.doc2bow(token) for token in all_tokens] 

#LDA 모델 학습
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=3, random_state=42, passes=10)

#토픽 출력
print(lda_model.print_topics(num_words=5))

[(0, '0.053*"자료" + 0.035*"금리" + 0.025*"리서치센터" + 0.024*"국고채" + 0.022*"상승"'), (1, '0.029*"CDS" + 0.024*"정책" + 0.021*"fed" + 0.019*"상황" + 0.016*"자료"'), (2, '0.022*"한은" + 0.017*"정부" + 0.014*"금리" + 0.014*"가능성" + 0.014*"물가"')]


In [36]:
#시각화
import pyLDAvis.gensim

pyLDAvis.enable_notebook() # added
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, sort_topics=False)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.160683  0.012009       1        1  36.871775
1      0.066537 -0.110841       2        1  29.750706
2      0.094146  0.098833       3        1  33.377519, topic_info=      Term       Freq      Total Category  logprob  loglift
49      자료  86.000000  86.000000  Default  30.0000  30.0000
339    CDS  32.000000  32.000000  Default  29.0000  29.0000
244  리서치센터  33.000000  33.000000  Default  28.0000  28.0000
44      한은  28.000000  28.000000  Default  27.0000  27.0000
85     국고채  32.000000  32.000000  Default  26.0000  26.0000
..     ...        ...        ...      ...      ...      ...
117     영향   7.827060  13.821463   Topic3  -5.0184   0.5287
28      정책  10.339242  38.718772   Topic3  -4.7401  -0.2231
87      인하   8.283520  18.208193   Topic3  -4.9617   0.3097
90      상승   8.052345  37.105946   Topic3  -4.9900  -0.4305
67      환율   7.454313  21.944004   Topic3  -5.0672   0.0176

[202 rows x 6 columns], token_table=      Topic      Freq Term
term                      
339       1  0.061031  CDS
339       2  0.915458  CDS
190       1  0.356015  GDP
190       3  0.623026  GDP
0         1  0.302640  fed
...     ...       ...  ...
75        3  0.199704   확대
67        1  0.637987   환율
67        3  0.318994   환율
263       1  0.981110  회사채
76        1  0.901344   흐름

[281 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])